In [1]:
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
import sys

In [2]:
load_dotenv('../config.env')

True

In [3]:
developer = os.getenv("DEVELOPER_NAME")
developer

'SUKANT KUMAR'

In [4]:
INPUT_DATA_FOLDER = os.path.join( os.path.abspath('../' ), 'data')
INPUT_DATA_FOLDER

'/Users/sukant.kumar/personal/codes/ml-model-deploy1/data'

## Import Data

In [5]:
IMAGE_DATA_FILE = 'train.csv.zip'
TEST_DATA_FILE = 'test.csv.zip'

In [6]:
data_file_path = os.path.join(INPUT_DATA_FOLDER,IMAGE_DATA_FILE)
data_file_path

'/Users/sukant.kumar/personal/codes/ml-model-deploy1/data/train.csv.zip'

In [7]:
images = pd.read_csv(data_file_path, compression='zip')

In [8]:
sys.getsizeof(images)/(1024 ** 2) #Size in MB

251.54127502441406

In [9]:
images.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [10]:
images.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
images.shape

(42000, 785)

In [12]:
images.label.unique() #10 different classes

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6])

In [13]:
images.label.value_counts()/images.shape[0] #All classes are roughly equally distributed

label
1    0.111524
7    0.104786
3    0.103595
9    0.099714
2    0.099452
6    0.098500
0    0.098381
4    0.096952
8    0.096738
5    0.090357
Name: count, dtype: float64

## The 10 classes are qually distributed. No need to use techniques to handle data imabalance

### Check for missing values

In [14]:
for column in images.columns:
    if images[column].isnull().sum() > 0:
        print('Column: ', column, ' has missing values: ', images[column].isnull().sum()/42000)

## No need for Missing Value Treatment

# Only need to scale values in a range of 0 to 1. The maximum value can pixel is taking is 255.
# Let's divide all inpute features by 255

## Strategy
## We'll fit 3 types of models
* 1 . skelarn based random forest model - for this we'll use GridSearch with CV as datasize is not very large
* 2 . Tensorflow based Feed Forward Neural Network model - for this we'll need to split data into train and test
* 3 . Tensorflow based Convolutional Neural Network model - for this we'll need to split data into train and test

## Steps of the pipeline
* Function: to read data from file, return dataframe
* Function: to scale input values in range of 0 to 1
* Function: to split data into train, test and validation set